In [1]:
import pandas as pd
import ssl
import urllib.request, urllib.parse, urllib.error
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import glob
import os.path
import re


In [2]:
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")


In [3]:
#rochelle.columns
namelist= []
for name in rochelle['Scientific Name']:
    namelist.append(name)

print(namelist)

['Cabomba caroliniana', 'Myriophyllum spicatum', 'Phragmites australis', 'Nymphoides peltata', 'Potamogeton crispus', 'Trapa natans', 'Corbicula fluminea', 'Gambusia affinis', 'Pimephales vigilax', 'Crassula helmsii', 'Pistia stratiotes', 'Stratiotes aloides', 'Veronica beccabunga', 'Rhithropanopeus harrisi', 'Thermocyclops crassus', 'Cherax destructor', 'Hypophthalmichthys molitrix', 'Hypophthalmichthys nobilis', 'Lepomis auritus', 'Limnoperna fortunei', 'Alnus glutinosa', 'Apeltes quadracus', 'Arundo donax', 'Branchiura sowerbyi', 'Channa argus', 'Cipangopaludina japonica', 'Cyclops strenuus', 'Didymosphenia geminata', 'Discostella (Cyclotella) pseudostelligera', 'Leuciscus idus', 'Neoergasilus japonicus', 'Procambarus clarkii', 'Psammonobiotus communis', 'Psammonobiotus dziwnowi', 'Psammonobiotus linearis', 'Syngnathus abaster', 'Agrostis gigantea', 'Butomus umbellatus', 'Carassius carassius', 'Carex acutiformis', 'Carex disticha', 'Chenopodium glaucum (Oxybasis glauca)', 'Cirsium p

In [4]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [5]:
#accessing search page of the herbarium
atlas_url = "https://bellatlas.umn.edu/collections/harvestparams.php"
atlas_html = urllib.request.urlopen(atlas_url, context=ctx).read()
#herbarium_html


In [8]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?
rounds = 0
masterdataframe = pd.DataFrame()
driver.maximize_window()
scraped_dataframe = pd.DataFrame()#(index=[0])
scraped_dataframe.reset_index(drop=True)
for speciesname in namelist[:]:
    #print(speciesname)
    driver.get(atlas_url) # Goes to atlas
    namebox = driver.find_element(by = By.XPATH, value = '//*[@id="taxa"]')
    searchbutton = driver.find_element(by = By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/form/div[2]/div[1]/button') #selects the advanced search button based off the pulled tag from right clicking the inspected element
    time.sleep(.5)
    namebox.clear()
    namebox.send_keys(speciesname) #'Cabomba caroliniana') #Sends in a name from Rochelle's list
    searchbutton.click()

    resultcount = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[4]/div[3]').text
    #print(resultcount) #Page 1, records 1-64 of 64
    resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results
    #print(resultcountnumberlist)
#if there are not results
    if resultcountnumberlist[1] == 0:
        continue
#continue to next search
    
#if there are results
    windowbefore = driver.window_handles[0]
    #resultcount = driver.find_element(by = By.XPATH, value ='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[4]/div[3]').text
    downloadbutton = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[1]/form/button')
    downloadbutton.click()
    windowafter = driver.window_handles[1]
    driver.switch_to.window(windowafter)
    time.sleep(1)
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[4]/td[2]/div/input[2]').click() #utf8 encoding
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[5]/td[2]/div/input').click() #uncompresses download

    downloadbutton=driver.find_element(by=By.NAME, value= "submitaction") #OVERWRITES THE PREVIOUS DOWNLOAD BUTTON
    downloadbutton.click()

    folder_path = r'C:\Users\justi\Downloads'
    file_type = r'\*CSV'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

    downloadeddata = pd.read_csv(max_file, encoding='utf-8')
    
    if masterdataframe.empty:
        masterdataframe = pd.DataFrame(data=downloadeddata)
        #print('Empty')
    else:
        masterdataframe = masterdataframe.append(pd.DataFrame(data = downloadeddata))
    driver.switch_to.window(windowbefore)
    rounds +=1
    

EmptyDataError: No columns to parse from file

In [7]:
masterdataframe

,id,institutionCode,collectionCode,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,otherCatalogNumbers,kingdom,phylum,...,verbatimDepth,verbatimElevation,disposition,language,recordEnteredBy,modified,sourcePrimaryKey-dbpk,collId,recordId,references
0,6318,MIN,Plants,NaN,PreservedSpecimen,8aa59e90-b0b1-4085-ac65-7d396f385597,911585,1134888,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:8aa59e90-b0b1-4085-ac65-7d396f385597,https://bellatlas.umn.edu:443/collections/indi...
1,7662,MIN,Plants,NaN,PreservedSpecimen,c459d154-b6d7-4eb8-a1db-91d79c8ee8b6,906278,1134887,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:c459d154-b6d7-4eb8-a1db-91d79c8ee8b6,https://bellatlas.umn.edu:443/collections/indi...
2,10966,MIN,Plants,NaN,PreservedSpecimen,a5d03247-7ffa-4f8e-af0b-be7e3210c806,906219,1134883,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:a5d03247-7ffa-4f8e-af0b-be7e3210c806,https://bellatlas.umn.edu:443/collections/indi...
3,16127,MIN,Plants,NaN,PreservedSpecimen,45527e00-3766-4b45-afff-93d99f5e90e1,477374,NaN,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:45527e00-3766-4b45-afff-93d99f5e90e1,https://bellatlas.umn.edu:443/collections/indi...
4,18640,MIN,Plants,NaN,PreservedSpecimen,903158b0-1dff-44a0-9785-01017e8b0744,427253,1134886,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:903158b0-1dff-44a0-9785-01017e8b0744,https://bellatlas.umn.edu:443/collections/indi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,1275847,MPCA,NaN,NaN,HumanObservation,a540b315-5eb1-4c10-bfad-7698153d170d,109300.0,NaN,Animalia,Chordata,...,NaN,NaN,NaN,NaN,NaN,2019-05-06 12:54:21,109300.0,16,urn:uuid:a540b315-5eb1-4c10-bfad-7698153d170d,https://bellatlas.umn.edu:443/collections/indi...
2489,1276031,MPCA,NaN,NaN,HumanObservation,d4a43537-75a7-47ea-8a94-eb2b24b9cfd2,109484.0,NaN,Animalia,Chordata,...,NaN,NaN,NaN,NaN,NaN,2019-05-06 12:54:21,109484.0,16,urn:uuid:d4a43537-75a7-47ea-8a94-eb2b24b9cfd2,https://bellatlas.umn.edu:443/collections/indi...
2490,1276054,MPCA,NaN,NaN,HumanObservation,10bff8e0-adc6-464e-8b5e-96a3f8b94545,109507.0,NaN,Animalia,Chordata,...,NaN,NaN,NaN,NaN,NaN,2019-05-06 12:54:21,109507.0,16,urn:uuid:10bff8e0-adc6-464e-8b5e-96a3f8b94545,https://bellatlas.umn.edu:443/collections/indi...
2491,1276276,MPCA,NaN,NaN,HumanObservation,ac17cd53-3c5e-4631-a93f-54adcb602a03,109729.0,NaN,Animalia,Chordata,...,NaN,NaN,NaN,NaN,NaN,2019-05-06 12:54:21,109729.0,16,urn:uuid:ac17cd53-3c5e-4631-a93f-54adcb602a03,https://bellatlas.umn.edu:443/collections/indi...
